In [47]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.svm import SVC
from numpy import mean
from numpy import std
import random as r
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV,KFold,RandomizedSearchCV

from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import balanced_accuracy_score, mean_squared_error
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import cross_validate
from sklearn.decomposition import PCA
from sklearn import tree
import copy
import warnings
warnings.filterwarnings("ignore")

In [80]:
def DecisionTree(Data,Target,k_fold,Data_test,Target_test,type):
    pipeline_tree = Pipeline([
    ('scaler', StandardScaler()),
    ('tree', tree.DecisionTreeClassifier())
    ])
    cv_inner = KFold(n_splits=5, shuffle=True, random_state=1)
    # define search space
    space = dict()
    space['tree__min_samples_leaf'] = list(range(50,200,25))
    space['tree__max_depth'] = list(range(10,60,10))
    space['tree__max_leaf_nodes'] = list(range(10,200,10))
    # define search
    search = RandomizedSearchCV(pipeline_tree, space, scoring='balanced_accuracy', n_jobs=1, cv=cv_inner, refit=True, n_iter = 100)
    # configure the cross-validation procedure
    cv_outer = KFold(n_splits=k_fold, shuffle=True, random_state=1)
    # execute the nested cross-validation
    scores_tree = cross_validate(search, Data, Target, cv=cv_outer, n_jobs=-1,return_estimator=True)
    #print(scores_tree['estimator'].best_params_)
    if type == "class":
        proba_tree = cross_val_predict(search, Data, Target, cv=cv_outer, n_jobs=-1,method = "predict_proba")
        proba_tree = proba_tree[:,1]
    else:
        proba_tree = cross_val_predict(search, Data, Target, cv=cv_outer, n_jobs=-1)
    scores__tree = cross_val_score(search, Data, Target, cv=cv_outer, n_jobs=-1).mean()
    parameters_tree = []
    for j in range(k_fold):
        parameters_tree.append(scores_tree['estimator'][j].best_params_)
    search.fit(Data, Target)
    preds_tree_test = search.predict(Data_test) 
    if type == "class":
        proba_tree_test = search.predict_proba(Data_test)[:,1]
        score_final_tree = balanced_accuracy_score(preds_tree_test,Target_test) 
    else: 
        proba_tree_test = preds_tree_test
        score_final_tree = mean_squared_error(preds_tree_test,Target_test)    
    return proba_tree, scores__tree, search.best_params_,proba_tree_test,score_final_tree

In [81]:
def Logistic_bruh_Regression(Data,Target,k_fold,Data_test,Target_test,type):
    pipeline_LR = Pipeline([
    ('scaler', StandardScaler()),
    ('LR', LogisticRegression(solver = "liblinear"))
    ])
    cv_inner = KFold(n_splits=5, shuffle=True, random_state=1)
    # define search space
    space = dict()
    space['LR__penalty'] = ["l1","l2"]
    space['LR__C'] = [0.01,0.1,0.3,0.5,1,5,10,100]
    # define search
    search = GridSearchCV(pipeline_LR, space, scoring='balanced_accuracy', n_jobs=1, cv=cv_inner, refit=True)
    # configure the cross-validation procedure
    cv_outer = KFold(n_splits=k_fold, shuffle=True, random_state=1)
    # execute the nested cross-validation
    scores_LR = cross_validate(search, Data, Target, cv=cv_outer, n_jobs=-1,return_estimator=True)
    #print(scores_tree['estimator'].best_params_)
    if type == "class":
        proba_LR = cross_val_predict(search, Data, Target, cv=cv_outer, n_jobs=-1,method = "predict_proba")
        proba_LR = proba_LR[:,1]
    else:
        proba_LR = cross_val_predict(search, Data, Target, cv=cv_outer, n_jobs=-1)
    scores__LR = cross_val_score(search, Data, Target, cv=cv_outer, n_jobs=-1).mean()
    parameters_LR = []
    for i in range(k_fold):
        parameters_LR.append(scores_LR['estimator'][i].best_params_)
    search.fit(Data, Target)
    preds_LR_test = search.predict(Data_test)
    if type == "class":
        proba_LR_test = search.predict_proba(Data_test)[:,1]
        score_final_LR = balanced_accuracy_score(preds_LR_test,Target_test) 
    else: 
        proba_LR_test = preds_LR_test
        score_final_LR = mean_squared_error(preds_LR_test,Target_test)
    return proba_LR, scores__LR, search.best_params_,proba_LR_test,score_final_LR

In [82]:
def LDA(Data,Target,k_fold,Data_test,Target_test):
    pipeline_LDA = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', LinearDiscriminantAnalysis())
    ])
    proba_LDA = cross_val_predict(pipeline_LDA, Data, Target, cv=k_fold, n_jobs=-1,method = "predict_proba")
    proba_LDA = proba_LDA[:,1]
    scores__LDA = cross_val_score(pipeline_LDA, Data, Target, cv=k_fold, n_jobs=-1).mean()
    pipeline_LDA.fit(Data,Target)
    proba_LDA_test = pipeline_LDA.predict_proba(Data_test)[:,1]
    preds_LDA_test = pipeline_LDA.predict(Data_test)
    score_final_LDA = balanced_accuracy_score(preds_LDA_test,Target_test)
    return proba_LDA,scores__LDA, proba_LDA_test,score_final_LDA

In [83]:
def QDA(Data,Target,k_fold,Data_test,Target_test):
    pipeline_QDA = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', QuadraticDiscriminantAnalysis())
    ])
    proba_QDA = cross_val_predict(pipeline_QDA, Data, Target, cv=k_fold, n_jobs=-1,method = "predict_proba")
    proba_QDA = proba_QDA[:,1] 
    scores__QDA = cross_val_score(pipeline_QDA, Data, Target, cv=k_fold, n_jobs=-1).mean()
    pipeline_QDA.fit(Data,Target)
    proba_QDA_test = pipeline_QDA.predict_proba(Data_test)[:,1]
    preds_QDA_test = pipeline_QDA.predict(Data_test)
    score_final_QDA = balanced_accuracy_score(preds_QDA_test,Target_test)
    return proba_QDA, scores__QDA,proba_QDA_test,score_final_QDA

In [84]:
def KNN(Data,Target,k_fold,Data_test,Target_test,type):
    pipeline_KNN = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
    ])
    cv_inner = KFold(n_splits=5, shuffle=True, random_state=1)
    # define search space
    space = dict()
    space['knn__n_neighbors'] = list(range(3,20,1))
    # define search
    search = GridSearchCV(pipeline_KNN, space, scoring='balanced_accuracy', n_jobs=1, cv=cv_inner, refit=True)
    # configure the cross-validation procedure
    cv_outer = KFold(n_splits=k_fold, shuffle=True, random_state=1)
    # execute the nested cross-validation
    scores_KNN = cross_validate(search, Data, Target, cv=cv_outer, n_jobs=-1,return_estimator=True)
    #print(scores_tree['estimator'].best_params_)
    if type == "class":
        proba_KNN = cross_val_predict(search, Data, Target, cv=cv_outer, n_jobs=-1,method = "predict_proba")
        proba_KNN = proba_KNN[:,1]
    else:
        proba_KNN = cross_val_predict(search, Data, Target, cv=cv_outer, n_jobs=-1)
    scores__KNN = cross_val_score(search, Data, Target, cv=cv_outer, n_jobs=-1).mean()
    parameters_KNN = []
    for i in range(k_fold):
        parameters_KNN.append(scores_KNN['estimator'][i].best_params_)
    search.fit(Data, Target)
    preds_KNN_test = search.predict(Data_test)
    if type == "class":
        proba_KNN_test = search.predict_proba(Data_test)[:,1]
        score_final_KNN = balanced_accuracy_score(preds_KNN_test,Target_test) 
    else: 
        proba_KNN_test = preds_KNN_test
        score_final_KNN = mean_squared_error(preds_KNN_test,Target_test)
    return proba_KNN,scores__KNN,search.best_params_,proba_KNN_test,score_final_KNN

In [85]:
def Stacking_Classifier_F(Data,Target,Models,type,k_fold= 5,again = 1,n_layers = 1,Data_test=None,Target_test=None,Meta_estimator="KNeighbors"):
    '''
    Jest to metoda uczenia maszynowego służąca do regresji oraz klasyfikacji binarnej (dla klas {-1,1}) typu ensemble, działający poprzez generowanie finalnych predykcji za
    pomocą meta estymatora, który uczy się przewidywania na podstawie zarówno danych wejściowych ORAZ predykcji z tych danych wygenerowanych przez metody składowe podane
    przez użytkownika za pomocą cross_val_predict.

    Parametry funkcji:
    Data - zbior treningowy (bez zmiennej celu)
    Target - treningowe "etykiety"
    Data_test - zbiór testowy
    Target_test - testowe "etykiety"
    type - typ problemu; dla klasyfikacji binarnej: "class", dla regresji: "reg"
    Models - lista zawierająca nazwy modeli użytych do stackingu, możliwe modele to: DecisionTreeClassifier,LogisticRegression,LDA,QDA,Knn, gdzie użytkownik podaje nazwy
    metod jako lista. Możliwe metody to "DecisionTree", "LogisticRegression","LDA","QDA","KNeighbors" i użytkownik umieszcza je na liście np. 
    ["DecisionTreeClassifier", "LogisticRegression"]. Uwaga! przy regresji nie można używać metod "LDA" oraz "QDA" 
    k_fold - liczność podziału w kroswalidacji (zewnętrznej, wewnętrzna kroswalidacja jest 5-krotna)
    n_layers - liczba warstw stackingu(domyślnie 1), np. 1,2,3,... - Uwaga! - n_layers >= 3 może być bardzo czasochłonne przy wywołaniu.
    again - zmienna mówiąca w ilu pierwszych warstwach stackingu użyć również danych treningowych oryginalnych (domyślnie 1), przyjmująca wartości 0,1,2,...,
    czyli again = 0 oznacza użycie danych tylko na samym początku, potem same predykcje, oczywiście again <= n_layers
    Meta_estimator - metoda używana do generowania finalnych predykcji przy użyciu wcześniejszych predykcji, możliwe metody to: Knn, Svm oraz średnia arytmetyczna, domyślnie Knn,
    wpisujemy je odpowiednio jako "KNeighbors", "SVM" lub "mean". Uwaga! dla regresji nie można stosować metody SVM 

    Funkcja zwraca listę zawierającą modele składowe oraz model finalny (stacking), ich balanced_accuracy_score lub (MSE dla regresji) na danych testowych 
    oraz hiperparametry(modeli składowych), gdzie modele składowe to modele oparte na danych BAZOWYCH, Uwaga! Dane są skalowane w funkcji, co zmienia MSE
    '''
    final_scores = []
    final_test_scores = []
    # Warstwy:
    Data_final_1 = copy.deepcopy(Data)
    Data_final_test = copy.deepcopy(Data_test)
    p = len(Data.axes[1])         
    for i in range(n_layers):   
        if "DecisionTree" in Models:
            proba_tree,scores__tree,parameters_tree,proba_tree_test,score_final_tree = DecisionTree(Data_final_1,Target,k_fold,Data_final_test,Target_test,type)
            Data_final_1[f'pred_tree_{i}'] = proba_tree.tolist()
            Data_final_test[f'pred_tree_{i}'] = proba_tree_test.tolist()
            final_scores.append(["Tree",scores__tree,i]) 
            final_test_scores.append(["Tree",score_final_tree,parameters_tree,i])
        if "Logistic_Regression" in Models:
            proba_LR,scores__LR,parameters_LR,proba_LR_test,score_final_LR = Logistic_bruh_Regression(Data_final_1,Target,k_fold,Data_final_test,Target_test,type)
            Data_final_1[f'pred_LR_{i}'] = proba_LR.tolist()
            Data_final_test[f'pred_LR_{i}'] = proba_LR_test.tolist() 
            final_scores.append(["LR",scores__LR,i])
            final_test_scores.append(["LR",score_final_LR,parameters_LR,i])
        if "LDA" in Models:
            proba_LDA,scores__LDA,proba_LDA_test,score_final_LDA = LDA(Data_final_1,Target,k_fold,Data_final_test,Target_test,type)
            Data_final_1[f'pred_LDA_{i}'] = proba_LDA.tolist() 
            Data_final_test[f'pred_LDA_{i}'] = proba_LDA_test.tolist()
            final_scores.append(["LDA",scores__LDA,i]) 
            final_test_scores.append(["LDA",score_final_LDA,"",i])           
        if "QDA" in Models:
            proba_QDA,scores__QDA,proba_QDA_test,score_final_QDA = QDA(Data_final_1,Target,k_fold,Data_final_test,Target_test,type)
            Data_final_1[f'pred_QDA_{i}'] = proba_QDA.tolist() 
            Data_final_test[f'pred_QDA_{i}'] = proba_QDA_test.tolist()
            final_scores.append(["QDA",scores__QDA,i])
            final_test_scores.append(["QDA",score_final_QDA,"",i])
        if "KNeighbors" in Models:
            proba_KNN,scores__KNN,parameters_KNN,proba_KNN_test,score_final_KNN = KNN(Data_final_1,Target,k_fold,Data_final_test,Target_test,type)
            Data_final_1[f'pred_KNN_{i}'] = proba_KNN.tolist() 
            Data_final_test[f'pred_KNN_{i}'] = proba_KNN_test.tolist()
            final_scores.append(["KNN",scores__KNN,i])
            final_test_scores.append(["KNN",score_final_KNN,parameters_KNN,i])
        if i == again:
            Data_final_1 =  Data_final_1.drop(Data_final_1.columns[range(p)],axis = 1)
            Data_final_test =  Data_final_test.drop(Data_final_test.columns[range(p)],axis = 1) 
    #final_scores.sort(key=lambda x: x[1]) 
    #best_model = [final_scores[-1][0],final_scores[-1][2]]
    final_test_scores_1 = []
    for i in final_test_scores:
        if i[3] == 0:
            final_test_scores_1.append(i)
    # Meta klasyfikator
    if Meta_estimator == "KNeighbors":        
        pipeline_meta = Pipeline([
        ('knn', KNeighborsClassifier())
        ])
        space = dict()
        space['knn__n_neighbors'] = list(range(8,25,1))
        search = GridSearchCV(pipeline_meta, space, scoring='balanced_accuracy', n_jobs=1, cv=5)
        search.fit(Data_final_1,Target)
        preds = search.predict(Data_final_test)
    if Meta_estimator == "SVM":        
        pipeline_meta = Pipeline([
        ('svm', SVC())
        ])
        space = dict()
        space['svm__C'] = [0.001,0.005,0.01,0.05,0.1,0.5,0,1,5,10,50,100]
        search = GridSearchCV(pipeline_meta, space, scoring='balanced_accuracy', n_jobs=1, cv=5)
        search.fit(Data_final_1,Target)
        preds = search.predict(Data_final_test) 
    if Meta_estimator == "mean":
        Data_final_test =  Data_final_test.drop(Data_final_test.columns[range(p)],axis = 1)
        Data_final_test['preds'] = Data_final_test.mean(axis=1)
        preds = Data_final_test['preds'].to_numpy()  
        preds = np.where(preds >= 0.5,1,-1)     
    if type == "class":
       final_test_scores_1.append(["Stack",balanced_accuracy_score(preds,Target_test),1])
    else:
       final_test_scores_1.append(["Stack",mean_squared_error(preds,Target_test),1])        
    final_test_scores_1.sort(key=lambda x: x[1],reverse=True)
    #acc = f"Stack accuracy: {balanced_accuracy_score(preds,Target_test)}"
    final_test_scores_1 = [final_test_scores_1[i][:-1] for i in range(len(final_test_scores_1))]
    return final_test_scores_1
    
    
        


           
    
            
    
      




       




        


In [8]:
X = pd.read_csv("C:\\Users\\User\\Desktop\\Python_5sem\\artifical_train_data.csv")
y = pd.read_csv("C:\\Users\\User\\Desktop\\Python_5sem\\artifical_train_labels.csv")
y = np.array(y)
y = y.flatten()
X = X.iloc[:,list(range(22,30,1))]
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2,shuffle=True)
values, counts = np.unique(y_train, return_counts=True)

In [86]:
Stacking_Classifier_F(Data = X_train,Target = y_train,Models =["DecisionTree","KNeighbors"],type = "class",again = 1,n_layers = 1,Data_test = X_test, Target_test=y_test,Meta_estimator="KNeighbors")

[['KNN', 0.8774647182464218, {'knn__n_neighbors': 5}],
 ['Stack', 0.8405253283302063],
 ['Tree',
  0.7433461034378466,
  {'tree__min_samples_leaf': 50,
   'tree__max_leaf_nodes': 10,
   'tree__max_depth': 50}]]

In [34]:
l = []
for i in range(10):
    X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2,shuffle=True)
    l.append(Stacking_Classifier_F(Data = X_train,Target = y_train,Models =["KNeighbors","DecisionTree"],again = 1,n_layers = 2,Data_test = X_test, Target_test=y_test))
l 

[0.8846779237023139,
 0.8477509949688369,
 0.8298936835522202,
 0.8798172277874017,
 0.8645903859174001,
 0.8447594436421677,
 0.8625090627265681,
 0.87508793086122,
 0.8616690488728402,
 0.8717653247826849]

In [35]:
ln = np.array(l)
ln.mean(),ln.std()

(0.8622521026813654, 0.016235538274587343)

In [ ]:
# meta classifier = KNN, classifiers = DecisionTree, 8 kolumn
# again = 1, n_layers = 1, rózne splity, srednia 0.85, sigma = 0.025
# again = 1, n_layers = 3, rózne splity, srednia 0.77, sigma = 0.022
# again = 3, n_layers = 3, rózne splity, srednia 0.84, sigma = 0.02

# meta classifier = KNN, classifiers = KNeighbors, 8 kolumn
# again = 1, n_layers = 1, rózne splity, srednia 0.84, sigma = 0.019
# again = 1, n_layers = 2, rózne splity, srednia 0.88, sigma = 0.018
# again = 1, n_layers = 3, rózne splity, srednia 0.89, sigma = 0.015
# again = 1, n_layers = 4, rózne splity, srednia 0.89, sigma = 0.011
# again = 1, n_layers = 5, rózne splity, srednia 0.89, sigma = 0.012

# meta classifier = KNN, classifiers = DecisionTree,KNeighbors, 8 kolumn
# again = 1, n_layers = 1, rózne splity, srednia 0.84, sigma = 0.015
# again = 1, n_layers = 2, rózne splity, srednia 0.86, sigma = 0.016